In [4]:
!pip install ~/qregress
# from qregress import *
import json, joblib
import os
import qregress
from qregress.settings import ANSATZ_LIST, ENCODER_LIST 

Processing /home/grierjones/qregress
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for qregress: filename=qregress-0.0.1-py3-none-any.whl size=63271 sha256=5dd1aae12216d05220f9401801c733c40718b8e249038c14a2a1640f1d11ec46
  Stored in directory: /tmp/pip-ephem-wheel-cache-1fj3ziw0/wheels/9f/40/a8/69c15ee9f03cf0d720ae4cf8d5bc231d3869acd7995f13a0f5
Successfully built qregress
  Attempting uninstall: qregress
    Found existing installation: qregress 0.0.1
    Uninstalling qregress-0.0.1:
      Successfully uninstalled qregress-0.0.1


In [5]:
def parse_settings(settings_file):
    with open(settings_file, 'r') as fp:
        settings = json.load(fp)

    global OPTIMIZER
    OPTIMIZER = settings['OPTIMIZER']

    global SHOTS
    SHOTS = settings['SHOTS']
                
    global BACKEND
    BACKEND = settings['BACKEND']

    global DEVICE
    DEVICE = settings['DEVICE']

    global SCALE_FACTORS
    SCALE_FACTORS = settings['SCALE_FACTORS']

    global POSTPROCESS
    POSTPROCESS = settings['POSTPROCESS']

    global ERROR_MITIGATION
    ERROR_MITIGATION = settings['ERROR_MITIGATION']

    global LAYERS
    LAYERS = settings['LAYERS']

    global HYPERPARAMETERS
    HYPERPARAMETERS = settings['HYPERPARAMETERS']
    # f was removed from HYPERPARAMETERS, this ensures old settings files can still run.
    if 'f' in HYPERPARAMETERS.keys():
        _ = HYPERPARAMETERS.pop('f', None)

    global RE_UPLOAD_DEPTH
    RE_UPLOAD_DEPTH = settings['RE-UPLOAD_DEPTH']

    global MAX_ITER
    MAX_ITER = settings['MAX_ITER']

    global TOLERANCE
    try:
        TOLERANCE = settings['TOLERANCE']
    except KeyError:
        TOLERANCE = None

    global NUM_QUBITS
    try:
        NUM_QUBITS = settings['NUM_QUBITS']
    except KeyError:
        NUM_QUBITS = None

    # classes aren't JSON serializable, so we store the key in the settings file and access it here.
    global ANSATZ
    ANSATZ = ANSATZ_LIST[settings['ANSATZ']]

    global ENCODER
    ENCODER = ENCODER_LIST[settings['ENCODER']]

    global BATCH_SIZE
    BATCH_SIZE = settings['BATCH_SIZE']
    
    global NUM_CORES
    NUM_CORES = settings['NUM_CORES']

In [6]:
def save_token(instance, token):
    global TOKEN
    TOKEN = token
    QiskitRuntimeService.save_account(channel='ibm_quantum', instance=instance, token=token, overwrite=True)

In [7]:
def create_kwargs():
    #  First have to apply specific ansatz settings: setting number of layers and the number of wires based on features
    ANSATZ.layers = LAYERS
    ANSATZ.set_wires(range(X_DIM))

    kwargs = {
        'encoder': ENCODER,
        'variational': ANSATZ,
        'num_qubits': X_DIM,
        'optimizer': OPTIMIZER,
        # 'optimizer': "BFGS",
        'max_iterations': MAX_ITER,
        'tol': TOLERANCE,
        'device': DEVICE,
        'shots': SHOTS,
        'backend': BACKEND,
        'postprocess': POSTPROCESS,
        'error_mitigation': ERROR_MITIGATION,
        'token': TOKEN,
        're_upload_depth': RE_UPLOAD_DEPTH,
        'batch_size': BATCH_SIZE,
        'njobs':NUM_CORES
    }
    return kwargs

In [8]:
def load_dataset(file):
    print(f'Loading dataset from {file}... ')
    data = joblib.load(file)
    X = data['X']
    y = data['y']

    global X_DIM
    _, X_DIM = X.shape
    print(f'Successfully loaded {file} into X and y data. ')
    return X, y

def save_token(instance, token):
    global TOKEN
    TOKEN = token
    QiskitRuntimeService.save_account(channel='ibm_quantum', instance=instance, token=token, overwrite=True)
    

In [9]:
parse_settings("A2_HWE-CNOT/A2_HWE-CNOT.json")

if DEVICE == 'qiskit.ibmq':
    save_token(instance, token)
else:
    TOKEN = None
global NUM_QUBITS
global X_DIM

if NUM_QUBITS is not None:
    X_DIM = NUM_QUBITS
elif X_DIM == 1:  # if X_DIM is None and num_qubits wasn't specified anywhere use a default value of 2.
    NUM_QUBITS = 2
    X_DIM = NUM_QUBITS

kwargs = create_kwargs()

In [10]:
X_train, y_train = load_dataset("0.1_5_DDCC_train.bin")

Loading dataset from 0.1_5_DDCC_train.bin... 
Successfully loaded 0.1_5_DDCC_train.bin into X and y data. 


In [13]:
model = qregress.quantum.QuantumRegressor(**kwargs)

12
12


In [14]:
model.fit(X_train, y_train)

Cost (Batches 38 of size 32): 100%|████████████████████████████| 38/38 [00:03<00:00,  9.99it/s]

[Mon Apr  7 20:41:03 2025]  Iteration number: 0 with current cost as 0.09442098773526543 and parameters 
[-2.90318345  2.23743464 -2.12427964 -0.11653103  0.55388708 -2.77010897
  3.06858498  2.18960145  1.18551998 -1.06648308  0.6027151   1.14432445
  1.31029899 -1.8735468   0.7296508   2.88578419 -0.54534335 -0.47522485
 -2.0265424   0.7289737   1.60512664  2.83077107 -1.2645671  -0.25136105
 -2.39279218 -2.27309774  3.13337155  2.54856958 -0.67550787 -2.69002202
  1.51701623 -0.04647228 -2.3427137   2.04043214  1.15216312 -0.65055499
  1.04465369 -1.92056016 -0.32067988  2.32226033 -0.20585092  0.56637433
  2.68983131  1.07273362  2.58102867  2.94254032  2.7651459  -2.94578571
  0.36684059 -0.65725907  1.65159086  2.68042142 -0.25194954 -2.45807906
  2.63570525 -3.07930429  0.79129724 -0.75901771 -1.81776562  2.9986886
 -1.85887169 -1.63734088 -2.55370824  2.30926632 -2.25921323  1.98533552
  1.90881014  1.42517315  2.59930885  2.87147469  1.84732041  1.52137428
 -1.69866533 -2.3443

array([-2.90318345,  2.23743464, -2.12427964, -0.11653103,  0.55388708,
       -2.77010897,  3.06858498,  2.18960145,  1.18551998, -1.06648308,
        0.6027151 ,  1.14432445,  1.31029899, -1.8735468 ,  0.7296508 ,
        2.88578419, -0.54534335, -0.47522485, -2.0265424 ,  0.7289737 ,
        1.60512664,  2.83077107, -1.2645671 , -0.25136105, -2.39279218,
       -2.27309774,  3.13337155,  2.54856958, -0.67550787, -2.69002202,
        1.51701623, -0.04647228, -2.3427137 ,  2.04043214,  1.15216312,
       -0.65055499,  1.04465369, -1.92056016, -0.32067988,  2.32226033,
       -0.20585092,  0.56637433,  2.68983131,  1.07273362,  2.58102867,
        2.94254032,  2.7651459 , -2.94578571,  0.36684059, -0.65725907,
        1.65159086,  2.68042142, -0.25194954, -2.45807906,  2.63570525,
       -3.07930429,  0.79129724, -0.75901771, -1.81776562,  2.9986886 ,
       -1.85887169, -1.63734088, -2.55370824,  2.30926632, -2.25921323,
        1.98533552,  1.90881014,  1.42517315,  2.59930885,  2.87